In [1]:
!pip install -r requirements.txt
!pip install git+https://github.com/openai/whisper.git 
!pip install yt-dlp
!pip install moviepy --upgrade
!apt-get update
!apt install imagemagick -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 17.4 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-uoafwlgz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-uoafwlgz
  Resolved https://github.com/openai/whisper.git to commit 9e653bd0ea0f1e9493cb4939733e9de249493cfb
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1174972 sha256=f0a6552beb478aa05506fdad2c2351ca0a7547a2fbaa51bbf5161aa42c2a0e79
  Stored in directory: /tmp/pip-ephem-wheel-cache-cy71trm1/wheels/fe/03/29/e7919208d11b4ab32972cb448bb84a9a675d92cd52c9a48341
Successfully built whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.3/111.3 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# remove line 88 of vim ~/../etc/ImageMagick-6/policy.xml
!sed -i '88d' ~/../etc/ImageMagick-6/policy.xml 
!mkdir experiments

mkdir: cannot create directory ‘t’: File exists


In [75]:
## Imports
from __future__ import unicode_literals
from yt_dlp import YoutubeDL
import yt_dlp
from IPython.display import Video
import whisper
import cv2
import pandas as pd
from moviepy.editor import VideoFileClip
import moviepy.editor as mp
from IPython.display import display, Markdown
from moviepy.editor import *
from moviepy.video.tools.subtitles import SubtitlesClip
import os

import cv2

def subtitle_video(download, url, aud_opts, vid_opts, model_type, name, audio_file, input_file, output, uploaded_vid = None):
#     params:
#     download: Default is True. This tells your function if you are downloading a youtube video
#     url: str, URL of youtube video
#     aud_opts: dict, audio youtube-dl options
#     vid_opts: dict, video youtube-dl options    
#     model_type: str, which pretrained model to download. options are ['tiny', 'small', 'base', 'medium','large']
#.    name: str, name of directory to store files in
#     audio_file: str, path to extracted audio file
#     input_file: str, path to video file
#     output: str, destination of final output
#     uploaded_vid: str, path to uploaded video file if download is False
#     
#--------------------------------------------------------
#   Returns: annotated video with translated captions into english
    
    ## First, Check if expermiment name is taken. If not, create the directory.
    ## Otherwise, you will be prompted to retry with a new name
    try:
        os.mkdir(f'experiments/{name}')
        print('Starting AutoCaptioning...')
        print(f'Results will be stored in experiments/{name}')
        
    except:
        return print('Choose another folder name! This one already has files in it.')
    
    ## Use video options for youtube-dl if downloading from youtube
    vid_opts['outtmpl'] = f'experiments/{name}/{input_file}'
    aud_opts['outtmpl'] = f'experiments/{name}/{audio_file}'

    URLS = [url]
    if download:
        with YoutubeDL(aud_opts) as ydl:
            ydl.download(url)
        with YoutubeDL(vid_opts) as ydl:
            ydl.download(URLS)
    else:
        # Use local clip if not downloading from youtube
        my_clip = mp.VideoFileClip(uploaded_vid)
        my_clip.audio.write_audiofile(f'experiments/{name}/{audio_file}')

    # Instantiate whisper model using model_type variable
    model = whisper.load_model(model_type)
    
    # Get text from speech for subtitles from audio file
    result = model.transcribe(f'experiments/{name}/{audio_file}', task = 'translate')
    
    # create Subtitle dataframe, and save it
    dict1 = {'start':[], 'end':[], 'text':[]}
    for i in result['segments']:
        dict1['start'].append(int(i['start']))
        dict1['end'].append(int(i['end']))
        dict1['text'].append(i['text'])
    df = pd.DataFrame.from_dict(dict1)
    df.to_csv(f'experiments/{name}/subs.csv')
    vidcap = cv2.VideoCapture(f'experiments/{name}/{input_file}')
    success,image = vidcap.read()
    height = image.shape[0]
    width =image.shape[1]

    # Instantiate MoviePy subtitle generator with TextClip, subtitles, and SubtitlesClip
    generator = lambda txt: TextClip(txt, font='P052-Bold', fontsize=width/20, stroke_width=1, color='white', stroke_color = 'black', size = (width, height-(3*height/4)), method='caption')
    # generator = lambda txt: TextClip(txt, color='white', fontsize=20, font='Georgia-Regular',stroke_width=3, method='caption', align='south', size=video.size)
    subs = tuple(zip(tuple(zip(df['start'].values, df['end'].values)), df['text'].values))
    subtitles = SubtitlesClip(subs, generator)
    
    # if the file was on youtube
    if download:
        video = VideoFileClip(f'experiments/{name}/{input_file}')
        final = CompositeVideoClip([video, subtitles.set_pos(('center','bottom'))])
        final.write_videofile(f'experiments/{name}/{output}', fps=video.fps, remove_temp=True, codec="libx264", audio_codec="aac")
    else:
        # If the file was a local upload
        video = VideoFileClip(uploaded_vid)
        final = CompositeVideoClip([video, subtitles.set_pos(('center','bottom'))])
        final.write_videofile(f'experiments/{name}/{output}', fps=video.fps, remove_temp=True, codec="libx264", audio_codec="aac")




In [76]:
## Declare relevant variables
# options for youtube download to ensure we get the right audio file.
# Only modify these if you need to. Lowered audio quality may inhibit the transcription's word error rate
opts_aud = {
    'format': 'mp3/bestaudio/best',
    'keep-video': True,
    # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
    'postprocessors': [{  # Extract audio using ffmpeg
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3', }]}
# options for youtube video to get right video file for final output
opts_vid = {'format': 'mp4/bestvideo/best'}

# Youtube URL
URL = 'https://youtu.be/3DKDt693p8Y'  # steamed hams sample link


In [77]:
# Generate subtitles

subtitle_video(
    download=True,
    uploaded_vid=None,  # path to local file
    url=URL,
    name='run1',
    aud_opts=opts_aud,
    vid_opts=opts_vid,  # Video download settings
    model_type='large',  # change to 'base' if your GPU has < 10 GB
    audio_file="audio.mp3",
    input_file='video.mp4',
    output='output.mp4')


Starting AutoCaptioning...
Results will be stored in experiments/run38
[youtube] 3DKDt693p8Y: Downloading webpage
[youtube] 3DKDt693p8Y: Downloading android player API JSON
[youtube] 3DKDt693p8Y: Downloading MPD manifest
[youtube] 3DKDt693p8Y: Downloading MPD manifest
[info] 3DKDt693p8Y: Downloading 1 format(s): 251
[download] Destination: experiments/run38/audio.mp3
[download] 100% of    5.79MiB in 00:00:00 at 10.17MiB/s  
[ExtractAudio] Destination: experiments/run38/audio.mp3
Deleting original file experiments/run38/audio.orig.mp3 (pass -k to keep)
[youtube] 3DKDt693p8Y: Downloading webpage
[youtube] 3DKDt693p8Y: Downloading android player API JSON
[youtube] 3DKDt693p8Y: Downloading MPD manifest
[youtube] 3DKDt693p8Y: Downloading MPD manifest
[info] 3DKDt693p8Y: Downloading 1 format(s): 22
[download] Destination: experiments/run38/video.mp4
[download] 100% of   84.63MiB in 00:00:02 at 34.26MiB/s     
Moviepy - Building video experiments/run38/output.mp4.
MoviePy - Writing audio in o

MoviePy - Done.
Moviepy - Writing video experiments/run38/output.mp4



Moviepy - Done !
Moviepy - video ready experiments/run38/output.mp4


<video controls src="experiments/run38/output.mp4" />
